In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Qi_padlock_sequence.csv')
df['Padlock'] = df['Padlock'].apply(lambda x: x.upper())
def simplify_name(n):
    if '-' not in n:
        return n
    else:
        return n.split('-')[1]
df['Gene'] = df['Name']#.apply(simplify_name)
df['Barcode'] = df['Padlock'].apply(lambda x: x[45:55])
print(len(df))
#df.to_csv('98_plex_primer_sequence_0930_simple.csv',index=False)
df.head()

58


,Name,Padlock,Gene,Barcode
0,Trh,CAGAGAAGAGGGGCCCCCACTCCCTACACGACGCTCTTCCGATCTG...,Trh,GCTCGGCTGG
1,Oxt,CTGCAGCAGATGCCTGTGGCTCCCTACACGACGCTCTTCCGATCTG...,Oxt,GTGTCGCGGG
2,Th,CCGGATGGTGTGAGGACTGTTCCCTACACGACGCTCTTCCGATCTG...,Th,GTTGGCGTCG
3,Sst,TCTGGTTGGGCTCGGACAGCTCCCTACACGACGCTCTTCCGATCTG...,Sst,GCTGGTGCGC
4,Crh,GCGCTGAGGCATCTGCAGCTTCCCTACACGACGCTCTTCCGATCTG...,Crh,GGCGTGTGCG


In [3]:
def overlap(seq_x,seq_y):
    output = ''
    for x,y in zip(seq_x,seq_y):
        if x == y:
            output += x
        elif 'G' not in x+y:
            output += 'A'
        else:
            if 'C' in x+y:
                output += 'C'
            elif 'T' in x+y:
                output += 'T'
    return output

plex_df = df[['Gene','Barcode']]
genes = list(df['Gene'])
barcodes = list(df['Barcode'])
plex_genes = []
plex_barcodes = []
for i in range(len(genes)):
    for j in range(i+1,len(genes)):
        plex_genes.append(f'{genes[i]}+{genes[j]}')
        plex_barcodes.append(overlap(barcodes[i],barcodes[j]))
temp_df = pd.DataFrame({'Gene':plex_genes,'Barcode':plex_barcodes})
plex_df = pd.concat([plex_df,temp_df])
#plex_df.to_csv('plex_map_0930.csv',index=False)
        

In [4]:
from tqdm import tqdm
def hamming(x,y):
    hamm = 0
    for a,b in zip(x,y):
        if a!=b:
            hamm += 1
    return hamm

def orthogonality(l,n):
    ambiguous = []
    d = {i:0 for i in range(0,len(l[0])+1)}
    for i in tqdm(range(len(l))):
        for j in range(i+1,len(l)):
            h = hamming(l[i],l[j])
            d[h] += 1
            if h <= 0:
                ambiguous.append(f'{l[i]},{l[j]}')
    return d,ambiguous

d,a = orthogonality(list(plex_df['Barcode']),list(plex_df['Gene']))

  0%|          | 0/1711 [00:00<?, ?it/s]

100%|██████████| 1711/1711 [00:01<00:00, 1135.75it/s]


In [5]:
components = list(set(a))
nodes = []
for c in components:
    nodes.append(c.split(',')[0])
    nodes.append(c.split(',')[0])
#print(set(nodes))

In [6]:
print(len(set(nodes)))

15


In [7]:
count = 0
for item in list(set(nodes)):
    if item in list(df['Barcode']):
        count += 1
print(count)

0


In [8]:
plex_df_filtered = plex_df[plex_df['Barcode'].apply(lambda x: x not in set(nodes))]
print(len(plex_df_filtered))

1680


In [9]:
d,a = orthogonality(list(plex_df_filtered['Barcode']),list(plex_df_filtered['Gene']))
print(d)

  0%|          | 0/1680 [00:00<?, ?it/s]

100%|██████████| 1680/1680 [00:01<00:00, 1156.08it/s]

{0: 0, 1: 437, 2: 4120, 3: 17286, 4: 50269, 5: 124614, 6: 245163, 7: 354861, 8: 350364, 9: 207906, 10: 55340}


In [10]:
plex_df_filtered.to_csv('Foodie_barcode.csv',index=False)

In [43]:
l = list(plex_df_filtered['Barcode'])
for c in ['A','T','C','G']:
    m = max([s.count(c) for s in l])
    print(c,m)

A 2
T 4
C 4
G 12


In [44]:
df1=plex_df_filtered[plex_df_filtered['Gene'].apply(lambda x:'+' not in x)]
df1.head()
print(len('GGGGGGGTGGGGGT'))

14
